#### installing libraries

In [1]:
!pip install mitreattack-python
!pip install ipywidgets

#### importing modules

In [2]:
import mitreattack.attackToExcel.attackToExcel as attackToExcel
import mitreattack.attackToExcel.stixToDf as stixToDf
from IPython.display import display, clear_output
from datetime import datetime
from ipywidgets import widgets
import json
import os
import pandas
import sys
import time

NumExpr defaulting to 8 threads.


#### collecting ATT&CK framework

In [3]:
attackdata = attackToExcel.get_stix_data("enterprise-attack")
techniques_data = stixToDf.techniquesToDf(attackdata, "enterprise-attack")
procedures_df = techniques_data["procedure examples"]
techniques_df = techniques_data["techniques"]

2023-06-29 20:59:33.729 | INFO     | mitreattack.attackToExcel.attackToExcel:get_stix_data:69 - Downloading ATT&CK data from github.com/mitre/cti
parsing relationships for type=technique: 100%|██████████| 16530/16530 [00:00<00:00, 33258.43it/s]


#### building widgets and obtaining selections

In [4]:
data = {}

output = widgets.Output()
spacer = widgets.Label("")

platform_label = widgets.Label("Platforms")
platform_input = widgets.Textarea()

procedure_label = widgets.Label("Threat Actors/Software")
procedure_input = widgets.Textarea()

keyword_label = widgets.Label("Keywords")
keyword_input = widgets.Textarea()

labels = widgets.VBox([platform_label, spacer, procedure_label, spacer, keyword_label])
inputs = widgets.VBox([platform_input, spacer, procedure_input, spacer, keyword_input])

combined_box = widgets.HBox([labels, inputs])

display(combined_box)

#### building dictionaries of correlated techniques and procedures

In [13]:
inputted_platforms = platform_input.value
inputted_procedures = procedure_input.value
inputted_keywords = keyword_input.value

yielded_techniques = {}
yielded_procedures = {}

if inputted_procedures != ".":
    inputted_procedures = list(str(procedure_input.value).split(","))
    for each_procedure in inputted_procedures:
        procedure = each_procedure.strip()
        for index, row in procedures_df.iterrows():
            if row["source name"] == procedure:
                if inputted_keywords != ".":
                    inputted_keywords = list(str(keyword_input.value).split(","))
                    for each_keyword in inputted_keywords:
                        keyword = each_keyword.strip()
                        if keyword in row["mapping description"]:
                            yielded_procedures["procedure_id"] = row["source ID"]
                            yielded_procedures["procedure_name"] = row["source name"]
                            yielded_procedures["procedure_description"] = row["mapping description"]
                            yielded_procedures["technique_id"] = row["target ID"]
                            yielded_procedures["technique_name"] = row["target name"]
                        else:
                            pass
                else:
                    yielded_procedures["procedure_id"] = row["source ID"]
                    yielded_procedures["procedure_name"] = row["source name"]
                    yielded_procedures["procedure_description"] = row["mapping description"]
                    yielded_procedures["technique_id"] = row["target ID"]
                    yielded_procedures["technique_name"] = row["target name"]
            else:
                pass
else:
    for index, row in procedures_df.iterrows():
        if inputted_keywords != ".":
            inputted_keywords = list(str(keyword_input.value).split(","))
            for each_keyword in inputted_keywords:
                keyword = each_keyword.strip()
                if keyword in row["mapping description"]:
                    yielded_procedures["procedure_id"] = row["source ID"]
                    yielded_procedures["procedure_name"] = row["source name"]
                    yielded_procedures["procedure_description"] = row["mapping description"]
                    yielded_procedures["technique_id"] = row["target ID"]
                    yielded_procedures["technique_name"] = row["target name"]
                else:
                    pass
        else:
            yielded_procedures["procedure_id"] = row["source ID"]
            yielded_procedures["procedure_name"] = row["source name"]
            yielded_procedures["procedure_description"] = row["mapping description"]
            yielded_procedures["technique_id"] = row["target ID"]
            yielded_procedures["technique_name"] = row["target name"]

In [23]:
for procedure_key, procedure_value in yielded_procedures.items():
    for index, row in techniques_df.iterrows():
        if yielded_procedures["technique_id"] == row["ID"]:
            if inputted_platforms != ".":
                inputted_platforms = list(str(platform_input.value).split(","))
                for each_platform in inputted_platforms:
                    if each_platform in row["platforms"]:
                        yielded_techniques["technique_id"] = row["ID"]
                        yielded_techniques["technique_name"] = row["name"]
                        yielded_techniques["tactics"] = row["tactics"]
                        yielded_techniques["data_sources"] = row["data sources"]
                        yielded_techniques["detection"] = row["detection"]
                        yielded_techniques["procedure_id"] = yielded_procedures["procedure_id"]
                        yielded_techniques["procedure_name"] = yielded_procedures["procedure_name"]
                        yielded_techniques["procedure_description"] = yielded_procedures["procedure_description"]
            else:
                yielded_techniques["technique_id"] = row["ID"]
                yielded_techniques["technique_name"] = row["name"]
                yielded_techniques["tactics"] = row["tactics"]
                yielded_techniques["data_sources"] = row["data sources"]
                yielded_techniques["detection"] = row["detection"]
                yielded_techniques["procedure_id"] = yielded_procedures["procedure_id"]
                yielded_techniques["procedure_name"] = yielded_procedures["procedure_name"]
                yielded_techniques["procedure_description"] = yielded_procedures["procedure_description"]



#### invoking extraction functions from MITRESaw.py